In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.cross_validation import train_test_split, KFold, cross_val_score
from sklearn import cross_validation
from sklearn import tree

In [2]:
training_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
print training_data
print test_data


                  datetime  season  holiday  workingday  weather   temp  \
0      2011-01-01 00:00:00       1        0           0        1   9.84   
1      2011-01-01 01:00:00       1        0           0        1   9.02   
2      2011-01-01 02:00:00       1        0           0        1   9.02   
3      2011-01-01 03:00:00       1        0           0        1   9.84   
4      2011-01-01 04:00:00       1        0           0        1   9.84   
5      2011-01-01 05:00:00       1        0           0        2   9.84   
6      2011-01-01 06:00:00       1        0           0        1   9.02   
7      2011-01-01 07:00:00       1        0           0        1   8.20   
8      2011-01-01 08:00:00       1        0           0        1   9.84   
9      2011-01-01 09:00:00       1        0           0        1  13.12   
10     2011-01-01 10:00:00       1        0           0        1  15.58   
11     2011-01-01 11:00:00       1        0           0        1  14.76   
12     2011-01-01 12:00:0

In [3]:
x_data = []
y_data = []
dates = {}
for i in training_data["count"]:
    y_data.append(i)
y_train = np.array(y_data)
for i in range(len(training_data["count"])):
    data = []
    for j in training_data:
        if j != "datetime" and j!= "count" and j!= "casual" and j!= "registered":
            data.append(training_data[j][i])
    data.append(int(training_data["datetime"][i][11:13]))
    npdata = np.array(data)
    x_data.append(npdata)
    dates[training_data["datetime"][i]] = npdata
print dates
x_train = np.array(x_data)

print x_train.shape
print y_train.shape
print x_train[0]
print y_train[0]
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=1338)
print len(X_train)
print len(Y_train)





{'2011-11-05 09:00:00': array([  4.    ,   0.    ,   0.    ,   1.    ,  12.3   ,  13.635 ,
        56.    ,  19.9995,   9.    ]), '2011-02-17 14:00:00': array([  1.    ,   0.    ,   1.    ,   1.    ,  24.6   ,  31.06  ,
        31.    ,  15.0013,  14.    ]), '2011-02-15 23:00:00': array([  1.    ,   0.    ,   1.    ,   2.    ,   9.02  ,  12.88  ,
        44.    ,   6.0032,  23.    ]), '2011-02-10 05:00:00': array([  1.    ,   0.    ,   1.    ,   2.    ,   4.92  ,   6.06  ,
        50.    ,  15.0013,   5.    ]), '2011-02-12 17:00:00': array([  1.    ,   0.    ,   0.    ,   1.    ,  13.12  ,  14.395 ,
        31.    ,  35.0008,  17.    ]), '2012-11-15 05:00:00': array([  4.    ,   0.    ,   1.    ,   2.    ,  12.3   ,  14.395 ,
        61.    ,  15.0013,   5.    ]), '2011-03-11 06:00:00': array([  1.    ,   0.    ,   1.    ,   1.    ,  10.66  ,  12.12  ,
        81.    ,  19.0012,   6.    ]), '2012-06-14 09:00:00': array([  2.    ,   0.    ,   1.    ,   1.    ,  25.42  ,  30.305 ,
      

In [4]:
est = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=7, random_state=0, loss='ls').fit(X_train, Y_train)



In [5]:
dtr = tree.DecisionTreeRegressor(max_depth=8)
dtr.fit(X_train, Y_train)
print dtr.predict(X_test)
print Y_test



[  99.2         207.35714286  171.5        ...,  180.31707317  207.35714286
  121.59090909]
[ 64 276 171 ..., 143 189 119]


In [6]:
def rmsle(y, y_pred):
    for i in range(len(y)):
        if y_pred[i] < 0:
            print "doesn't work"
    assert len(y) == len(y_pred)
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

#taken from https://www.kaggle.com/marknagelberg/caterpillar-tube-pricing/rmsle-function
#NOTE: when using this function with the Gradient Boosting regressor, we saw unexpected behavior: namely, the GBR returning negative values when those were not possible in this data set.
#To mitigate this, we've set all negative values returned to 0.

In [10]:
print len(dtr.predict(X_test))
print len(Y_test)
print rmsle(Y_test, dtr.predict(X_test))

2178
2178
0.49733777689


In [11]:
est_pred = est.predict(X_test)
for i in range(len(est_pred)):
    if est_pred[i] < 0:
        est_pred[i] = 0
print rmsle(Y_test, est_pred)

0.464693140324


In [10]:
for i in X_test:
    print i

[  2.       0.       0.       1.      14.76    18.18    71.       7.0015]
[  4.      0.      1.      1.      9.84   11.365  48.     12.998]
[  2.       0.       1.       3.      25.42    29.545   78.      23.9994]
[  2.      0.      1.      3.     27.06   30.305  83.     12.998]
[  1.       0.       1.       1.       8.2      8.335   27.      31.0009]
[  2.     0.     1.     1.    20.5   24.24  51.     0.  ]
[  2.       0.       1.       2.      21.32    25.      83.       8.9981]
[  3.       0.       1.       1.      24.6     28.03    83.       7.0015]
[  1.     0.     0.     1.    10.66  15.15  41.     0.  ]
[  2.       0.       0.       3.      17.22    21.21    41.      15.0013]
[  1.       0.       1.       1.      12.3     15.15    52.      11.0014]
[  4.       0.       1.       1.      13.94    17.425   81.       6.0032]
[  2.     0.     1.     2.    17.22  21.21  77.     0.  ]
[  4.       0.       1.       2.      13.12    15.15    49.      22.0028]
[  3.       0.       1.     